# MUSEO VIRTUAL INTERACTIVO: ART AFTER DARK

### Creación de la base da datos e inserción de datos

In [123]:
# Importar librerías para manipulación y análisis de datos
import mysql.connector
from mysql.connector import errorcode
import requests
import pandas as pd
import numpy as np
import re


In [ ]:
#Abrimos los DataFrames:

df_spotify_csv = pd.read_csv("spoty_min.csv", index_col=False)
df_lastfm_csv = pd.read_csv("lastfm_min.csv", index_col=False)

pd.set_option('display.max_rows', None) #para que muestre todos los registros

In [125]:
df_spotify_csv.sample(5)

,name_track,name_artist,genre,type,year,popularity,total_tracks,disc_number,duration_ms,explicit
5087,cartier santos,sdm,pop,track,2024,78,14,1,183813,True
4941,alibi (feat. rudimental),ella henderson,pop,track,2024,76,1,1,184852,False
5149,llevame a bailar,embusteros,rock,track,2024,48,10,1,217018,False
2971,brain revolution girl 2016 ver.,maretu,rock,track,2016,59,15,1,220108,False
4948,magnetic,illit,pop,track,2024,84,4,1,160688,False


In [126]:
df_lastfm_csv.sample(5)

,artista,de_gira,reproducciones,oyentes,biografia,similares,genero
977,ella henderson,0,7250749,616669,Ella Henderson (born 12 January 1996) is a Bri...,"becky hill, jess glynne, leona lewis, mimi web...","pop, british, soul, singer-songwriter, x factor"
90,mj lenderman,1,8701351,272864,MJ Lenderman is the solo project of singer-son...,"wednesday, greg freeman, merce lemon, silver j...","alt-country, indie rock, slowcore, slacker roc..."
1040,chimbala,0,650894,72520,,"don miguelo, bulin 47, el alfa, rochy rd, arca...","reggaeton, dominican republic, dominican, dembow"
456,caleb hearn,0,1549512,128412,Caleb Hearn (born 23. june 2000) was born in t...,"matt hansen, suriel hess, anson seabra, rosie ...","singer-songwriter, pop, folk, indie pop, indie..."
33,barns courtney,0,8413049,445923,Barns Courtney (born Barnaby George Courtney o...,"the unlikely candidates, des rocs, oh the larc...","seen live, alternative, folk, singer-songwrite..."


In [107]:
df_similars.sample(5)

,artist,similar_artist
4235,kygo,gryffin
3668,meghan trainor,zara larsson
680,kevin rudolf,timbaland
2485,kaaris,sch
3812,electric guest,slenderbodies


In [108]:
df_genres.sample(5)

,artist,genre
6367,erre xi,rnb
779,david civera,eurovision
5262,gorillaz,hip-hop
3451,rat boy,punk
3031,babymetal,metal


In [109]:
# ELIMINACIÓN

cnx = mysql.connector.connect(user='root', password = 'AlumnaAdalab', host='127.0.0.1')
mycursor = cnx.cursor()

try:
    mycursor.execute('DROP DATABASE IF EXISTS musium')
    print(mycursor)

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: DROP DATABASE IF EXISTS musium


In [110]:
# Creación del Schema

cnx = mysql.connector.connect(user='root', password = 'AlumnaAdalab', host='127.0.0.1')
mycursor = cnx.cursor()

try:
    mycursor.execute('CREATE DATABASE IF NOT EXISTS musium')
    print(mycursor)

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: CREATE DATABASE IF NOT EXISTS musium


In [111]:
# Creación de la tabla de Spotify
cnx = mysql.connector.connect(user='root', password = 'AlumnaAdalab', host='127.0.0.1')
mycursor = cnx.cursor()

try:
    mycursor.execute('USE musium')
    mycursor.execute('''
                     CREATE TABLE IF NOT EXISTS tracks (
                     id_track INT AUTO_INCREMENT PRIMARY KEY, 
                     track_name VARCHAR(320), 
                     artist_name VARCHAR(100),
                     genre VARCHAR(15), 
                     type VARCHAR(8),
                     year YEAR,
                     popularity INT, 
                     total_tracks INT, 
                     disc_number INT, 
                     duration_ms FLOAT,
                     explicit VARCHAR(10) )
                     ''') 
    print(mycursor)

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: 
                     CREATE TABLE IF NO..


In [ ]:
# Convierte las celdas vacías en NULL 

df_spotify_csv = df_spotify_csv.replace({np.nan: None})

In [115]:
# Carga de la tabla spotify en SQL

order = "INSERT INTO tracks (track_name, artist_name, genre, type, year, popularity, total_tracks, disc_number, duration_ms, explicit) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
values = df_spotify_csv.values.tolist()

try:
    mycursor.executemany(order, values)
    cnx.commit()
    print(mycursor.rowcount, "registros insertados.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

6000 registros insertados.


In [116]:
# Creación tabla LASTFM

cnx = mysql.connector.connect(user='root', password = 'AlumnaAdalab', host='127.0.0.1', database = 'musium')
mycursor = cnx.cursor()

try:
    mycursor.execute('''
                     CREATE TABLE IF NOT EXISTS artist (
                     artist_name VARCHAR(100) PRIMARY KEY, 
                     on_tour VARCHAR(10), 
                     playcount BIGINT, 
                     listeners INT, 
                     biography TEXT, 
                     similar VARCHAR(380), 
                     genres VARCHAR(180))
                     ''') 
    print(mycursor)

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: 
                     CREATE TABLE IF NO..


In [ ]:
# Convierte las celdas vacías en NULL 

df_lastfm_csv = df_lastfm_csv.replace({np.nan: None})

In [119]:
# Inserción de valores de la tabla LAST FM
cnx = mysql.connector.connect(user='root', password = 'AlumnaAdalab', host='127.0.0.1', database = 'musium')
mycursor = cnx.cursor()

order_2 = "INSERT INTO artist (artist_name, on_tour, playcount, listeners, biography, similar, genres) VALUES (%s,%s,%s,%s,%s,%s,%s)"
values_2 = df_lastfm_csv.values.tolist()

try:
    mycursor.executemany(order_2, values_2)
    cnx.commit()
    print(mycursor.rowcount, "registros insertados.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1506 registros insertados.


In [ ]:
# Establece artist_name como FOREIGN KEY

cnx = mysql.connector.connect(user='root', password = 'AlumnaAdalab', host='127.0.0.1', database = 'musium')
mycursor = cnx.cursor()

try:
    mycursor.execute('ALTER TABLE tracks ADD FOREIGN KEY (artist_name) REFERENCES artist(artist_name) ON DELETE CASCADE ON UPDATE CASCADE')
    print(mycursor)

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: ALTER TABLE tracks ADD FOREIGN KEY (arti..


In [120]:
cnx = mysql.connector.connect(user='root', password = 'AlumnaAdalab', host='127.0.0.1', database = 'musium')
mycursor = cnx.cursor()

try:
    mycursor.execute('')
    print(mycursor)

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: (Nothing executed yet)


In [128]:
# CIERRA CONEXIÓN

cnx.close()